In [ ]:
cd ..

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch

from neural_nets_library import training
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.tree_decoder import TreeDecoder
from tree_to_sequence.program_datasets import IdentityTreeToTreeDataset
from tree_to_sequence.tree_to_tree import TreeToTree



In [ ]:
num_vars = 10
num_ints = 11

for_ops = {
    "Var": 0,
    "Const": 1,
    "Plus": 2,
    "Minus": 3,
    "EqualFor": 4,
    "LeFor": 5,
    "GeFor": 6,
    "Assign": 7,
    "If": 8,
    "Seq": 9,
    "For": 10
}

for_ops = {"<" + k.upper() + ">": v for k,v in for_ops.items()}

lambda_calculus_ops = {
                "<VARIABLE>": 0,
                "<ABSTRACTION>": 1,
                "<NUMBER>": 2,
                "<BOOLEAN>": 3,
                "<NIL>": 4,
                "<IF>": 5,
                "<CONS>": 6,
                "<MATCH>": 7,
                "<UNARYOPER>": 8,
                "<BINARYOPER>": 9,
                "<LET>": 10,
                "<LETREC>": 11,
                "<TRUE>": 12,
                "<FALSE>": 13,
                "<TINT>": 14,
                "<TBOOL>": 15,
                "<TINTLIST>": 16,
                "<TFUN>": 17,
                "<ARGUMENT>": 18,
                "<NEG>": 19,
                "<NOT>": 20,
                "<PLUS>": 21,
                "<MINUS>": 22,
                "<TIMES>": 23,
                "<DIVIDE>": 24,
                "<AND>": 25,
                "<OR>": 26,
                "<EQUAL>": 27,
                "<LESS>": 28,
                "<APPLICATION>": 29,
                "<HEAD>": 30,
                "<TAIL>": 31
            }

In [ ]:
use_embedding = True
binarize = True
num_ints = 5
num_vars = len(for_ops)

In [ ]:
identity_dset = IdentityTreeToTreeDataset("ANC/Easy-arbitraryForListWithOutput.json", 
                                      binarize=binarize, is_lambda_calculus=False, 
                                      num_ints=num_ints, num_vars=num_vars,
                                      use_embedding=use_embedding, cuda=False)


In [ ]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform(param, -stdev, stdev)

In [ ]:
embedding_size = 30
hidden_size = 40
num_layers = 1
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops.keys())
encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2], attention=True, use_embedding=use_embedding)

max_num_children = 2

decoder = TreeDecoder(embedding_size, hidden_size, max_num_children, for_ops, align_type=align_type)
program_model = TreeToTree(encoder, decoder, hidden_size, embedding_size, 
                           alignment_size=alignment_size, align_type=align_type)
    
reset_all_parameters_uniform(program_model, 0.1)
encoder.initialize_forget_bias(3)
decoder.initialize_forget_bias(3)

In [ ]:
#program_model = program_model.cuda()

In [ ]:
optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=100, factor=0.8)

In [ ]:
training.train_model_tree_to_tree(program_model, identity_dset, 
                                 optimizer, lr_scheduler=lr_scheduler, num_epochs=3, 
                                 batch_size=100, plateau_lr=True, print_every=100,
                                 use_cuda=False)